In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()
sc = spark.sparkContext

In [6]:
rdd = sc.wholeTextFiles('./tiny_files')
rdd.map(lambda x:x[1]).collect()

['hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink',
 'hello spark\r\nhello hadoop\r\nhello flink']